In [1]:
from utils.signal import EMGSignal, Signal, SignalPlotter
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json

import matplotlib.pylab as pylab
params = {
    'legend.fontsize': 'x-large',
    'figure.figsize': (24, 6),
    'axes.labelsize': 'x-large',
    'axes.titlesize':'x-large',
    'xtick.labelsize':'x-large',
    'ytick.labelsize':'x-large'
}
pylab.rcParams.update(params)

In [2]:
def read_data(datapath: str, fs: int = None) -> tuple[dict, list[Signal]]:
    df = pd.read_excel(f"{datapath}.xlsx")
    values = df.to_numpy()
    with open(f"{datapath}_points.json", "r") as f:
        points = json.load(f)
    
    return (
        points,
        [
            EMGSignal(values[:, 0], values[:, 1], fs=fs),
            EMGSignal(values[:, 0], values[:, 2], fs=fs),
            EMGSignal(values[:, 0], values[:, 3], fs=fs),
        ]
    )

In [ ]:
DATA_PATH = "data/1/1min/exp_1"
FS = 1000

pt, signals = read_data(DATA_PATH, FS)

for signal in signals:
    signal.baseline(pt["baseline"]).bandpass(20, 450).notch()
    SignalPlotter.psd(signal)
    signal.lowpass(40).crop(pt["close_0"]-0.01, pt["open_2"] + 0.01)
time = signals[0].x

In [ ]:
plt.figure(figsize=(24, 6))
plt.plot(time, signals[0].y, label="EMG 1", alpha=0.7, color="blue")
plt.plot(time, signals[1].y, label="EMG 2", alpha=0.7, color="gray")
plt.plot(time, signals[2].y, label="EMG 3", alpha=0.7, color="purple")

periods = []
c = ["r", "g"]
for i in range(3):
    t0 = np.argwhere(time >= pt[f"close_{i}"])[0][0]
    tf = np.argwhere(time >= pt[f"open_{i}"])[0][0]
    periods.extend([t0, tf])
for i in range(len(periods) - 1):
    if i in [0, 1]:
        plt.axvspan(
            time[periods[i]],
            time[periods[i + 1]],
            color=c[i % 2],
            alpha=0.3,
            label=f"{'Sitdown action' if i % 2 == 0 else 'Stand up action'}",
        )
    else:
        plt.axvspan(
            time[periods[i]],
            time[periods[i + 1]],
            color=c[i % 2],
            alpha=0.3,
        )
plt.xlabel("Time (seconds)")
plt.ylabel("Amplitude (voltes)")
plt.legend()
plt.grid(linewidth=1, linestyle="--", alpha=0.7)
plt.show()